In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np

In [10]:
# Load data
train_df = pd.read_csv('dataset/sign_mnist_train.csv')
test_df = pd.read_csv('dataset/sign_mnist_test.csv')

In [11]:
# Prepare the data
class SignLanguageDataset(Dataset):
    def __init__(self, df):
        self.X = df.iloc[:, 1:].values.reshape(-1, 1, 28, 28).astype(np.float32) / 255.0
        self.y = df.iloc[:, 0].values
        self.y = np.eye(26)[self.y].astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

train_dataset = SignLanguageDataset(train_df)
test_dataset = SignLanguageDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

In [12]:
# Define the CNN model
class SignLanguageCNN(nn.Module):
    def __init__(self):
        super(SignLanguageCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(4, 4)
        self.fc1 = nn.Linear(16 * 3 * 3, 128)
        self.fc2 = nn.Linear(128, 26)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16 * 3 * 3)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # No sigmoid here; use BCEWithLogitsLoss instead
        return x

In [13]:
# Initialize model, loss function, and optimizer
model = SignLanguageCNN()
criterion = nn.BCEWithLogitsLoss()  # Combines sigmoid with BCELoss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
# Train the model
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/100], Loss: {running_loss/len(train_loader):.4f}')

    # Evaluate training accuracy
    model.eval()
    train_correct = 0
    train_total = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs = model(inputs)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            train_total += labels.size(0) * labels.size(1)  # Account for multi-label
            train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    print(f'Training Accuracy: {train_accuracy:.2f}%')

Epoch [1/100], Loss: nan
Training Accuracy: 96.15%
Epoch [2/100], Loss: nan
Training Accuracy: 96.15%
Epoch [3/100], Loss: nan
Training Accuracy: 96.15%
Epoch [4/100], Loss: nan
Training Accuracy: 96.15%
Epoch [5/100], Loss: nan
Training Accuracy: 96.15%
Epoch [6/100], Loss: nan
Training Accuracy: 96.15%
Epoch [7/100], Loss: nan
Training Accuracy: 96.15%
Epoch [8/100], Loss: nan
Training Accuracy: 96.15%
Epoch [9/100], Loss: nan
Training Accuracy: 96.15%
Epoch [10/100], Loss: nan
Training Accuracy: 96.15%
Epoch [11/100], Loss: nan
Training Accuracy: 96.15%
Epoch [12/100], Loss: nan
Training Accuracy: 96.15%
Epoch [13/100], Loss: nan
Training Accuracy: 96.15%
Epoch [14/100], Loss: nan
Training Accuracy: 96.15%
Epoch [15/100], Loss: nan
Training Accuracy: 96.15%
Epoch [16/100], Loss: nan
Training Accuracy: 96.15%
Epoch [17/100], Loss: nan
Training Accuracy: 96.15%
Epoch [18/100], Loss: nan
Training Accuracy: 96.15%
Epoch [19/100], Loss: nan
Training Accuracy: 96.15%
Epoch [20/100], Loss:

In [15]:
# Evaluate the model on the test set
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        test_total += labels.size(0) * labels.size(1)  # Account for multi-label
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 96.15%
